In [1]:
import pandas as pd
import json
import requests

In [2]:
RS_type = {
    "0": "RSU",
    "1": "RSIA",
    "2": "RSK GM",
    "3": "RSK BEDAH",
    "4": "RSK Mata",
    "5": "RSK Jiwa",
    "6": "RSK JANTUNG",
    "7": "RSK KANKER",
    "8": "RSK THT-KL",
    "9": "RSK OTAK",
    "10": "RSK INFEKSI",
    "11": "RSKO"
}

In [3]:
response = requests.get('http://eis.dinkes.jakarta.go.id/eis/home/ajax_list_data_bed', headers={'accept': 'application/json'})

In [4]:
j = json.loads(response.text[response.text.index("{"):response.text.index("}")+1])

In [5]:
bor_header = ['No', 'Wilayah', 'Nama RS', 'Ketersediaan ICU Tekanan Negatif Dengan Ventilator',
                  'Ketersediaan ICU Tekanan Negatif Tanpa Ventilator', 'Ketersediaan ICU Tanpa Tekanan Negatif dengan Ventilator',
                  'Ketersediaan ICU Tanpa Tekanan Negatif tanpa Ventilator', 'Ketersediaan Isolasi Tekanan Negatif',
                  'Ketersediaan Isolasi Tanpa Tekanan Negatif', 'Ketersediaan NICU khusus COVID-19', 'Ketersediaan Perina khusus COVID-19',
                  'Ketersediaan PICU khusus COVID-19', 'Ketersediaan OK khusus COVID-19', 'Ketersediaan HD khusus COVID-19',
                  'Update Terakhir', 'Hotline SPGDT']

curl_df = pd.DataFrame.from_dict(j['data'])
curl_df.columns = bor_header

In [6]:
def get_availabilility_columns(df):
    return [col for col in df.columns if 'Ketersediaan' in col]

In [7]:
curl_df[get_availabilility_columns(curl_df)] = curl_df[get_availabilility_columns(curl_df)].apply(pd.to_numeric)

In [8]:
rs_clean_df = pd.read_csv('rs_dki_cleaned.csv', dtype={"Latitude": str, "Longitude": str})

In [9]:
curl_df['Nama RS'] = curl_df['Nama RS'].apply(lambda x: 'RS Darurat Covid Wisma Atlet' if 'RSDC' in x else x)
curl_df['Nama RS'] = curl_df['Nama RS'].apply(lambda x: 'RS Ibu dan Anak Avisena' if 'Avisena' in x else x)

combined_df = pd.merge(curl_df,rs_clean_df,on='Nama RS')

In [10]:
# If need key to access detail, hospital_id_as_index=True and change orient='index' in .to_json() below
hospital_id_as_index = False
if hospital_id_as_index:
    combined_df = combined_df.set_index('Hospital ID')

In [11]:
from geopy.distance import geodesic

def calc_distance(row, site_coords):
    station_coords = (row['Latitude'], row['Longitude'])
    dist = geodesic(site_coords, station_coords).km
    return(dist)

In [12]:
# Change "0" with desired variable
combined_df = combined_df[combined_df["Jenis RS"] == RS_type["0"]]
combined_df["Jenis RS"].value_counts()

RSU    141
Name: Jenis RS, dtype: int64

In [13]:
# Change latitude & latitude with desired variables
tes_lat = -6.317546231739774
tes_lon = 106.94013742650952
combined_df['Jarak'] = combined_df.apply(calc_distance, site_coords=(tes_lat, tes_lon), axis=1)
combined_df['Jarak_Weighted'] = combined_df['Jarak'].copy()

In [14]:
availability_cols = get_availabilility_columns(combined_df)
combined_df['Total Ketersediaan'] = combined_df[availability_cols].sum(axis=1)
combined_df['Ketersediaan_Weighted'] = combined_df['Total Ketersediaan'].copy()

In [15]:
# Encode 'Kelas RS'
combined_df.loc[combined_df.query("Kelas == 'A'").index, 'Kelas_Weighted'] = 5
combined_df.loc[combined_df.query("Kelas == 'B'").index, 'Kelas_Weighted'] = 4
combined_df.loc[combined_df.query("Kelas == 'C'").index, 'Kelas_Weighted'] = 3
combined_df.loc[combined_df.query("Kelas == 'D'").index, 'Kelas_Weighted'] = 2
combined_df.loc[combined_df.query("Kelas == 'Belum Ditetapkan'").index, 'Kelas_Weighted'] = 1

In [16]:
cols = ['Ketersediaan_Weighted','Kelas_Weighted','Jarak_Weighted']
for col in cols:
  combined_df[col] = (combined_df[col]-combined_df[col].min())/(combined_df[col].max()-combined_df[col].min())

In [17]:
combined_df['Jarak_Weighted'] = 1 - combined_df['Jarak_Weighted']
combined_df.loc[combined_df.query("Ketersediaan_Weighted == 0").index, 'Ketersediaan_Weighted'] = -1

In [18]:
combined_df['Rank'] = (combined_df['Jarak_Weighted']*0.45 + combined_df['Kelas_Weighted']*0.1 + combined_df['Ketersediaan_Weighted']*0.45).rank(ascending=False).astype('int')
combined_df = combined_df.sort_values("Rank")

In [19]:
final_features = ['Kode RS', 'Nama RS', 'Wilayah',
       'Total Ketersediaan', 'Ketersediaan ICU Tekanan Negatif Dengan Ventilator',
       'Ketersediaan ICU Tekanan Negatif Tanpa Ventilator',
       'Ketersediaan ICU Tanpa Tekanan Negatif dengan Ventilator',
       'Ketersediaan ICU Tanpa Tekanan Negatif tanpa Ventilator',
       'Ketersediaan Isolasi Tekanan Negatif',
       'Ketersediaan Isolasi Tanpa Tekanan Negatif',
       'Ketersediaan NICU khusus COVID-19',
       'Ketersediaan Perina khusus COVID-19',
       'Ketersediaan PICU khusus COVID-19', 'Ketersediaan OK khusus COVID-19',
       'Ketersediaan HD khusus COVID-19', 'Hotline SPGDT',
       'Jenis RS', 'Kelas',
       'Jarak', 'Image URL', 'Rank']

In [20]:
# Change 10 with desired variable
if hospital_id_as_index:
    final_data = combined_df[final_features][:10]
else:
    final_data = combined_df[['Hospital ID']+final_features][:10]

In [21]:
# If need key to access detail, change hospital_id_as_index=True above and orient='index'
# If not, orient='records'
if hospital_id_as_index:
    result = final_data.to_json(orient='index')
else:
    result = final_data.to_json(orient='records')
parsed = json.loads(result)
parsed


[{'Hospital ID': '41898d5654984230bb6dbcede4d56f9d',
  'Kode RS': '3171900S',
  'Nama RS': 'RS Darurat Covid Wisma Atlet',
  'Wilayah': 'Jakarta Pusat',
  'Total Ketersediaan': 1859,
  'Ketersediaan ICU Tekanan Negatif Dengan Ventilator': 0,
  'Ketersediaan ICU Tekanan Negatif Tanpa Ventilator': 0,
  'Ketersediaan ICU Tanpa Tekanan Negatif dengan Ventilator': 0,
  'Ketersediaan ICU Tanpa Tekanan Negatif tanpa Ventilator': 1859,
  'Ketersediaan Isolasi Tekanan Negatif': 0,
  'Ketersediaan Isolasi Tanpa Tekanan Negatif': 0,
  'Ketersediaan NICU khusus COVID-19': 0,
  'Ketersediaan Perina khusus COVID-19': 0,
  'Ketersediaan PICU khusus COVID-19': 0,
  'Ketersediaan OK khusus COVID-19': 0,
  'Ketersediaan HD khusus COVID-19': 0,
  'Hotline SPGDT': '119 ext 9',
  'Jenis RS': 'RSU',
  'Kelas': 'A',
  'Jarak': 20.4036209327,
  'Image URL': 'https://asset.kompas.com/crops/ccE8j44xe0jsKKiVO1ey5Sdvbc8=/0x0:0x0/750x500/data/photo/2020/03/19/5e73605448212.jpg',
  'Rank': 1},
 {'Hospital ID': '8c1